In [1]:
import numpy as np
from shared.simulators import ConstantEffectSimulator, ModeratedEffectSimulator, DataSimulator, EffectSimulatorCollection
from shared.analyzers import AnalyzerCollection, WLSAnalyzer, IPWAnalyzer, SIPWAnalyzer, MetaAnalyzer
from shared.result_processors import Results
import matplotlib.pyplot as plt

In [2]:
N = 50
T = 100
alpha = 0.05
n_reps = 100
weight_type = "ATE"

effect_simulator = ModeratedEffectSimulator("Noisy Moderated Effects", N, 1., 1., 0.3, 1., 0.2, epsilon_sd=0.5)
effect_simulator.sample_params()
data_simulator = DataSimulator(N, T).simulate_history()
data_simulator = data_simulator.simulate_effects(effect_simulator)
data_simulator = data_simulator.simulate_a()

In [3]:
y1_1 = 1.2
y1_0 = -0.3

y2_11 = 0.7
y2_10 = 0.7
y2_01 = -0.8
y2_00 = 2.4

p1 = 0.6
p2_1 = 0.7
p2_0 = 0.8

In [4]:
z1 = p2_1 * y2_11 + (1. - p2_1) * y2_10
z0 = p2_0 * y2_01 + (1. - p2_0) * y2_00
tau = z1 - z0

In [5]:
Var_E_tauhat = ((1.-p1)*z1 + p1*z0)**2 / (p1 * (1. - p1))
Var1 = (p2_1 * (1.-p2_1)) * (y2_11 - y2_10)**2
Var0 = (p2_0 * (1.-p2_0)) * (y2_01 - y2_00)**2
E_Var_tauhat = Var1/p1 + Var0/(1.-p1)
Var_tauhat = E_Var_tauhat + Var_E_tauhat

In [6]:
nreps = 1_000_000
tauhats = np.zeros(nreps)
for i in range(nreps):
    a1 = np.random.random() < p1
    p1_a1 = p1 if a1 else 1. - p1
    p2_a1 = p2_1 if a1 else p2_0
    a2 = np.random.random() < p2_a1
    if a1:
        if a2:
            y = y2_11
        else:
            y = y2_10
    else:
        if a2:
            y = y2_01
        else:
            y = y2_00        
    sign = 1. if a1 else -1.    
    tauhats[i] = sign * y / p1_a1
    
mean_tauhat = np.mean(tauhats)
var_tauhat = np.var(tauhats)
print(f"tau: {tau:.3f}")
print(f"Sample mean: {mean_tauhat:.3f}")
print(f"Var(tauhat): {Var_tauhat:.3f}")
print(f"Sample variance: {var_tauhat:.3f}")

tau: 0.860
Sample mean: 0.859
Var(tauhat): 4.237
Sample variance: 4.238
